In [71]:
import os
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
# from src.wilson import * 
from src.models import * 
import matplotlib.pyplot as plt
nk.config.netket_debug = True
import numpy as np

## Parameters

In [38]:
L = 3

note hilbert space is on edges
there is no natural ordering to the hilbert space so we will have to impose one.
this is specified by wilson.ordered_edges.
as long as we respect this ordering across all implementations, things should work. Convention is vertical links first.

In [48]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 18, LxLx2 = 18


The Z_2 gauge theory Hamiltonian is 
$$
H = -K \sum_{\square} \prod_{ij \in \square} \sigma_{ij}^z - g \prod_{ij} \sigma_{ij}^x
$$

In [52]:
def z2_hamiltonian(theta, g):
	h = 0
	edges = ordered_edges(g)

	for i in g.nodes(): 
		l, r, u, d = plaquette_indices(i, L)
		h += - np.cos(theta) * sigmaz(hi, l) * sigmaz(hi, d) * sigmaz(hi, r) * sigmaz(hi, u)

	for i in range(g.n_edges):
		h += - np.sin(theta) * sigmax(hi, i)

	return h

## Vary parameters and verify against ED 

In [80]:
thetas = np.linspace(0, np.pi/2, 10)
exact_energies = [] 
genn_energies = []
genn_energies_var = []

i = 1
for theta in thetas:
	print(f'{i}/{len(thetas)}')

	h = z2_hamiltonian(theta, g)
	exact_energy = nk.exact.lanczos_ed(h)
	print(f'exact energy:\t{exact_energy}')

	genn = GENN(graph=g)
	sampler = nk.sampler.MetropolisLocal(hi)
	vstate = nk.vqs.MCState(sampler, model=genn, n_samples=1008)
	optimizer = nk.optimizer.Sgd(learning_rate=0.05)
	preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
	vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
	log = nk.logging.RuntimeLog()
	vmc.run(n_iter=500, out=log)
	
	genn_energy = vstate.expect(h)
	
	exact_energies.append(exact_energy)
	genn_energies.append(genn_energy.mean.real)
	genn_energies_var.append(genn_energy.variance)
	i += 1

1/10
exact energy:	[-9.]


100%|██████████| 500/500 [00:09<00:00, 51.03it/s, Energy=-9.000e+00+0.000e+00j ± nan [σ²=0.000e+00]]     


2/10
exact energy:	[-9.02006245]


100%|██████████| 500/500 [00:15<00:00, 32.97it/s, Energy=-8.993+0.001j ± 0.016 [σ²=0.247, R̂=1.0116]]               


3/10
exact energy:	[-9.34285914]


...

100%|██████████| 500/500 [00:13<00:00, 37.79it/s, Energy=-17.05388+0.00000j ± 0.00076 [σ²=0.00059, R̂=1.0118]]


9/10
exact energy:	[-17.76099136]


100%|██████████| 500/500 [00:13<00:00, 38.42it/s, Energy=-17.76060+0.00000j ± 0.00035 [σ²=0.00012, R̂=1.0016]]


10/10
exact energy:	[-18.]


100%|██████████| 500/500 [00:12<00:00, 39.47it/s, Energy=-18.000010-0.000009j ± 0.000013 [σ²=0.000000, R̂=1.0058]]


In [82]:
genn_energies = np.array(genn_energies)
genn_energies_var = np.array(genn_energies_var)
thetas = np.array(thetas)
exact_energies = np.array(exact_energies)

In [83]:
np.savetxt(f'../data/z2_varying_h_{L}x{L}.txt', np.column_stack((thetas, exact_energies, genn_energies, genn_energies_var)))

## Confining Phase Transition

In [85]:
L = 8

In [ ]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

In [ ]:
def wilson_loop:

In [ ]:
h = z2_hamiltonian(theta, g)

genn = GENN(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=genn, n_samples=1008)
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=500, out=log)